In [1]:
import numpy as np
import pandas as pd
from py_vollib_vectorized import get_all_greeks
import plotly as py
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.offline import init_notebook_mode
init_notebook_mode(connected = True)

In [ ]:
def plot():
    fig = make_subplots(4,1)
    x = np.arange(500,1500,100)

    # trace0 = go.Scatter(x = x,
    #                 y = pnl,
    #                 mode='lines',
    #                 name='到期pnl')
    trace1 = go.Scatter(x = x,
                    y = delta,
                    mode='lines',
                    name='delta')
    trace2 = go.Scatter(x = x,
                    y = gamma,
                    mode='lines',
                    name='gamma')
    trace3 = go.Scatter(x = x,
                    y = theta,
                    mode='lines',
                    name='theta')
    trace4 = go.Scatter(x = x,
                    y = vega,
                    mode='lines',
                    name='vega')
    # fig.add_trace(trace0,1,1)
    fig.add_trace(trace1,1,1)
    fig.add_trace(trace2,2,1)
    fig.add_trace(trace3,3,1)
    fig.add_trace(trace4,4,1)

    py.offline.iplot(fig)

In [ ]:
# 牛市看涨
# call：平值多头 + 虚值空头

s = 1000
k1,k2 = 1000,1500
t = 0.5
sigma = 0.3

# pnl
pnl = []
for i in range(500,2000,100):
    pnl1 = (i - k1 ) if i > k1 else 0
    pnl2 = 0 if i < k2 else (k2 - i)
    pl = pnl1 + pnl2
    pnl.append(pl)

# delta
delta=[]
# Gamma, Theta, Vega
gamma,theta,vega = [],[],[]
for i in range(500,2000,100):
    delta.append(get_all_greeks('c',i,k1,t,0,sigma)['delta'][0] - get_all_greeks('c',i,k2 ,t,0,sigma)['delta'][0])
    gamma.append(get_all_greeks('c',i,k1,t,0,sigma)['gamma'][0] - get_all_greeks('c',i,k2 ,t,0,sigma)['gamma'][0])
    theta.append(get_all_greeks('c',i,k1,t,0,sigma)['theta'][0] - get_all_greeks('c',i,k2 ,t,0,sigma)['theta'][0])
    vega.append(get_all_greeks('c',i,k1,t,0,sigma)['vega'][0] - get_all_greeks('c',i,k2 ,t,0,sigma)['vega'][0])

plot()

In [ ]:
# 熊市看跌
# put：平值多头 + 虚值空头

s = 1000
k1,k2 = 1000,500
t = 0.5
sigma = 0.3

# pnl
pnl = []
for i in range(500,2000,100):
    pnl1 = (k1 -i) if i < k1 else 0
    pnl2 = 0 if i > k2 else (i-k2)
    pl = pnl1 + pnl2
    pnl.append(pl)

# delta
delta=[]
# Gamma, Theta, Vega
gamma,theta,vega = [],[],[]
for i in range(500,2000,100):
    delta.append(get_all_greeks('p',i,k1,t,0,sigma)['delta'][0] - get_all_greeks('p',i,k2 ,t,0,sigma)['delta'][0])
    gamma.append(get_all_greeks('p',i,k1,t,0,sigma)['gamma'][0] - get_all_greeks('p',i,k2 ,t,0,sigma)['gamma'][0])
    theta.append(get_all_greeks('p',i,k1,t,0,sigma)['theta'][0] - get_all_greeks('p',i,k2 ,t,0,sigma)['theta'][0])
    vega.append(get_all_greeks('p',i,k1,t,0,sigma)['vega'][0] - get_all_greeks('p',i,k2 ,t,0,sigma)['vega'][0])

plot()

In [ ]:
# 宽跨式多头
s = 1000
k1,k2 = 800,1200
t = 0.5
sigma = 0.3

# pnl
pnl = []
for i in range(500,2000,100):
    pnl1 = (i - k1) if i > k1 else 0
    pnl2 = (k2-i) if i < k2 else 0
    pl = pnl1 + pnl2
    pnl.append(pl)

# Gamma, Theta, Vega
delta, gamma, theta, vega = [],[],[],[]
for i in range(500,2000,100):
    delta.append(get_all_greeks('c',i,k1,t,0,sigma)['delta'][0] + get_all_greeks('p',i,k2 ,t,0,sigma)['delta'][0])
    gamma.append(get_all_greeks('c',i,k1,t,0,sigma)['gamma'][0] + get_all_greeks('p',i,k2 ,t,0,sigma)['gamma'][0])
    theta.append(get_all_greeks('c',i,k1,t,0,sigma)['theta'][0] + get_all_greeks('p',i,k2 ,t,0,sigma)['theta'][0])
    vega.append(get_all_greeks('c',i,k1,t,0,sigma)['vega'][0] + get_all_greeks('p',i,k2 ,t,0,sigma)['vega'][0])

plot()

# pnl分析
由于对冲掉了delta，pnl变为
$$Pnl =\frac{1}{2}\Gamma(RV^2 - \sigma_{IV}^2) + Vega\Delta_{\sigma_{IV}^2}$$

可以看出，主要的风险敞口在于Gamma与Vega，以及对冲间隔时间内的Delta

对于目前的做市策略，只是控制Delta敞口，在大部分时候都会形成Gamma与Vega的暴露，目前只能**不断通过买卖价差来赚取流动性收益**，来弥补时间价值的损耗以及IV下降带来的潜在风险

但这样会产生一个问题，比如在做市过程中，如果市场整体持仓偏向看多，按原有做市逻辑，做市账户净头寸会偏空方，说明做市商会被动形成对波动率的看法，如果市场发生不利变动则会产生大量亏损


In [ ]:
# 跨式空头
s = 1000
k1,k2 = 800,1200
t = 0.5
sigma = 0.3

# pnl
pnl = []
for i in range(500,2000,100):
    pnl1 = -(i - k1) if i > k1 else 0
    pnl2 = -(k2 - i) if i < k2 else 0
    pl = pnl1 + pnl2
    pnl.append(pl)

# Gamma, Theta, Vega
delta, gamma, theta, vega = [],[],[],[]
for i in range(500,2000,100):
    delta.append(-get_all_greeks('c',i,k1,t,0,sigma)['delta'][0] - get_all_greeks('p',i,k2 ,t,0,sigma)['delta'][0])
    gamma.append(-get_all_greeks('c',i,k1,t,0,sigma)['gamma'][0] - get_all_greeks('p',i,k2 ,t,0,sigma)['gamma'][0])
    theta.append(-get_all_greeks('c',i,k1,t,0,sigma)['theta'][0] - get_all_greeks('p',i,k2 ,t,0,sigma)['theta'][0])
    vega.append(-get_all_greeks('c',i,k1,t,0,sigma)['vega'][0] - get_all_greeks('p',i,k2 ,t,0,sigma)['vega'][0])

plot()

In [ ]:
# 跨式多头 + 日历价差多头
# long t1 put + short t2 put + long t1 call + short t2 call

s = 1000
k1,k2 = 800,1200
t1,t2 = 0.5,0.8
sigma = 0.3


# Gamma, Theta, Vega
delta, gamma, theta, vega = [],[],[],[]
for i in range(500,2000,100):
    delta.append(get_all_greeks('c',i,k1,t1,0,sigma)['delta'][0] + get_all_greeks('p',i,k2 ,t1,0,sigma)['delta'][0] -
                 get_all_greeks('c',i,k1,t2,0,sigma)['delta'][0] - get_all_greeks('p',i,k2 ,t2,0,sigma)['delta'][0])

    gamma.append(get_all_greeks('c',i,k1,t1,0,sigma)['gamma'][0] + get_all_greeks('p',i,k2 ,t1,0,sigma)['gamma'][0] -
                 get_all_greeks('c',i,k1,t2,0,sigma)['gamma'][0] - get_all_greeks('p',i,k2 ,t2,0,sigma)['gamma'][0])

    theta.append(get_all_greeks('c',i,k1,t1,0,sigma)['theta'][0] + get_all_greeks('p',i,k2 ,t1,0,sigma)['theta'][0] -
                 get_all_greeks('c',i,k1,t2,0,sigma)['theta'][0] - get_all_greeks('p',i,k2 ,t2,0,sigma)['theta'][0])

    vega.append(get_all_greeks('c',i,k1,t1,0,sigma)['vega'][0] + get_all_greeks('p',i,k2 ,t1, 0, sigma)['vega'][0] -
                get_all_greeks('c',i,k1,t2,0,sigma)['vega'][0] - get_all_greeks('p',i,k2 ,t2, 0, sigma)['vega'][0])

plot()

In [ ]:
# 蝶式多头
# long ITM call + short OTM call X2 + long OTM call

s = 1000
k1,k2,k3 = 800,1000,1200
t = 0.5
sigma = 0.3

# pnl
pnl = []
for i in range(500,2000,100):
    pnl1 = (i - k1) if i > k1 else 0
    pnl2 = 0 if i < k2 else (k2-i) * 2
    pnl3 = (i - k3) if i > k3 else 0
    pl = pnl1 + pnl2 + pnl3
    pnl.append(pl)

# delta
delta=[]
# Gamma, Theta, Vega
gamma,theta,vega = [],[],[]
for i in range(500,2000,100):
    delta.append(get_all_greeks('c',i,k1,t,0,sigma)['delta'][0] - get_all_greeks('c',i,k2 ,t,0,sigma)['delta'][0] * 2 + get_all_greeks('c',i,k3,t,0,sigma)['delta'][0])
    gamma.append(get_all_greeks('c',i,k1,t,0,sigma)['gamma'][0] - get_all_greeks('c',i,k2 ,t,0,sigma)['gamma'][0] * 2 + get_all_greeks('c',i,k3,t,0,sigma)['gamma'][0])
    theta.append(get_all_greeks('c',i,k1,t,0,sigma)['theta'][0] - get_all_greeks('c',i,k2 ,t,0,sigma)['theta'][0] * 2 + get_all_greeks('c',i,k3,t,0,sigma)['theta'][0])
    vega.append(get_all_greeks('c',i,k1,t,0,sigma)['vega'][0] - get_all_greeks('c',i,k2 ,t,0,sigma)['vega'][0] * 2 + get_all_greeks('c',i,k3,t,0,sigma)['vega'][0])

plot()

In [ ]:
# 蝶式空头
# short ITM call + long ATM call X2 + short OTM call

s = 1000
k1,k2,k3 = 800,1000,1200
t = 0.5
sigma = 0.3

# pnl
pnl = []
for i in range(500,2000,100):
    pnl1 = (k1 - i) if i > k1 else 0
    pnl2 = (i - k2) * 2 if i > k2 else 0
    pnl3 = (k3 - i) if i > k3 else 0
    pl = pnl1 + pnl2 + pnl3
    pnl.append(pl)

# delta
delta=[]
# Gamma, Theta, Vega
gamma,theta,vega = [],[],[]
for i in range(500,2000,100):
    delta.append(-get_all_greeks('c',i,k1,t,0,sigma)['delta'][0] + get_all_greeks('c',i,k2 ,t,0,sigma)['delta'][0] * 2 - get_all_greeks('c',i,k3,t,0,sigma)['delta'][0])
    gamma.append(-get_all_greeks('c',i,k1,t,0,sigma)['gamma'][0] + get_all_greeks('c',i,k2 ,t,0,sigma)['gamma'][0] * 2 - get_all_greeks('c',i,k3,t,0,sigma)['gamma'][0])
    theta.append(-get_all_greeks('c',i,k1,t,0,sigma)['theta'][0] + get_all_greeks('c',i,k2 ,t,0,sigma)['theta'][0] * 2 - get_all_greeks('c',i,k3,t,0,sigma)['theta'][0])
    vega.append(-get_all_greeks('c',i,k1,t,0,sigma)['vega'][0] + get_all_greeks('c',i,k2 ,t,0,sigma)['vega'][0] * 2 - get_all_greeks('c',i,k3,t,0,sigma)['vega'][0])

plot()

In [ ]:
# condor
# long k1 put + short k2 put + short k3 call + long k4 call

s = 1000
k1,k2,k3,k4 = 800,1000,1200,1400
t = 0.5
sigma = 0.3

# pnl
pnl = []
for i in range(500,2000,100):
    pnl1 = (k1 - i) if i < k1 else 0
    pnl2 = (i - k2) if i < k2 else 0
    pnl3 = (k3 - i) if i > k3 else 0
    pnl4 = (i - k4) if i > k4 else 0
    pl = pnl1 + pnl2 + pnl3 + pnl4
    pnl.append(pl)

delta, gamma, theta, vega = [],[],[],[]
for i in range(500,2000,100):
    delta.append(get_all_greeks('p',i,k1,t,0,sigma)['delta'][0] - get_all_greeks('p',i,k2 ,t,0,sigma)['delta'][0] - get_all_greeks('c',i,k3 ,t,0,sigma)['delta'][0] + get_all_greeks('c',i,k4 ,t,0,sigma)['delta'][0])

    gamma.append(get_all_greeks('p',i,k1,t,0,sigma)['gamma'][0] - get_all_greeks('p',i,k2 ,t,0,sigma)['gamma'][0] - get_all_greeks('c',i,k3,t,0,sigma)['gamma'][0] + get_all_greeks('c',i,k4,t,0,sigma)['gamma'][0])

    theta.append(get_all_greeks('p',i,k1,t,0,sigma)['theta'][0] - get_all_greeks('p',i,k2 ,t,0,sigma)['theta'][0] - get_all_greeks('c',i,k3 ,t,0,sigma)['theta'][0] + get_all_greeks('c',i,k4 ,t,0,sigma)['theta'][0])

    vega.append(get_all_greeks('p',i,k1,t,0,sigma)['vega'][0] - get_all_greeks('p',i,k2 ,t,0,sigma)['vega'][0] - get_all_greeks('c',i,k3 ,t,0,sigma)['vega'][0] + get_all_greeks('c',i,k4 ,t,0,sigma)['vega'][0])

plot()

In [ ]:
# condor空头
s = 1000
k1,k2,k3,k4 = 800,1000,1200,1400
t = 0.5
sigma = 0.3

# pnl
pnl = []
for i in range(500,2000,100):
    pnl1 = -(k1 - i) if i < k1 else 0
    pnl2 = -(i - k2) if i < k2 else 0
    pnl3 = -(k3 - i) if i > k3 else 0
    pnl4 = -(i - k4) if i > k4 else 0
    pl = pnl1 + pnl2 + pnl3 + pnl4
    pnl.append(pl)

delta, gamma, theta, vega = [],[],[],[]
for i in range(500,2000,100):
    delta.append(-get_all_greeks('p',i,k1,t,0,sigma)['delta'][0] + get_all_greeks('p',i,k2 ,t,0,sigma)['delta'][0] + get_all_greeks('c',i,k3 ,t,0,sigma)['delta'][0] - get_all_greeks('c',i,k4 ,t,0,sigma)['delta'][0])

    gamma.append(-get_all_greeks('p',i,k1,t,0,sigma)['gamma'][0] + get_all_greeks('p',i,k2 ,t,0,sigma)['gamma'][0] + get_all_greeks('c',i,k3,t,0,sigma)['gamma'][0] - get_all_greeks('c',i,k4,t,0,sigma)['gamma'][0])

    theta.append(-get_all_greeks('p',i,k1,t,0,sigma)['theta'][0] + get_all_greeks('p',i,k2 ,t,0,sigma)['theta'][0] + get_all_greeks('c',i,k3 ,t,0,sigma)['theta'][0] - get_all_greeks('c',i,k4 ,t,0,sigma)['theta'][0])

    vega.append(-get_all_greeks('p',i,k1,t,0,sigma)['vega'][0] + get_all_greeks('p',i,k2 ,t,0,sigma)['vega'][0] + get_all_greeks('c',i,k3 ,t,0,sigma)['vega'][0] - get_all_greeks('c',i,k4 ,t,0,sigma)['vega'][0])

plot()

In [ ]:
# time butterfly

# 时间蝶式空头
# short ITM call + long ATM call X2 + short OTM call

s = 1000
k1,k2,k3 = 800,1000,1200
t1,t2,t3 = 0.5,0.6,0.7
sigma = 0.3

# pnl
pnl = []
for i in range(500,2000,100):
    pnl1 = (k1 - i) if i > k1 else 0
    pnl2 = (i - k2) * 2 if i > k2 else 0
    pnl3 = (k3 - i) if i > k3 else 0
    pl = pnl1 + pnl2 + pnl3
    pnl.append(pl)

# delta
delta=[]
# Gamma, Theta, Vega
gamma,theta,vega = [],[],[]
for i in range(500,2000,100):
    delta.append(-get_all_greeks('c',i,k1,t1,0,sigma)['delta'][0] + get_all_greeks('c',i,k2 ,t2,0,sigma)['delta'][0] * 2 - get_all_greeks('c',i,k3,t3,0,sigma)['delta'][0])
    gamma.append(-get_all_greeks('c',i,k1,t1,0,sigma)['gamma'][0] + get_all_greeks('c',i,k2 ,t2,0,sigma)['gamma'][0] * 2 - get_all_greeks('c',i,k3,t3,0,sigma)['gamma'][0])
    theta.append(-get_all_greeks('c',i,k1,t1,0,sigma)['theta'][0] + get_all_greeks('c',i,k2 ,t2,0,sigma)['theta'][0] * 2 - get_all_greeks('c',i,k3,t3,0,sigma)['theta'][0])
    vega.append(-get_all_greeks('c',i,k1,t1,0,sigma)['vega'][0] + get_all_greeks('c',i,k2 ,t2,0,sigma)['vega'][0] * 2 - get_all_greeks('c',i,k3,t3,0,sigma)['vega'][0])

plot()

In [7]:
# 跨式多头 + 日历价差多头
# long t1 put + short t2 put + long t1 call + short t2 call

s = 1000
k1,k2 = 800,1200
t1,t2 = 0.5,0.8
sigma = 0.3


# Gamma, Theta, Vega
delta, gamma, theta, vega = [],[],[],[]
for i in range(500,2000,100):
    delta.append(get_all_greeks('c',i,k1,t1,0,sigma)['delta'][0] + get_all_greeks('p',i,k2 ,t1,0,sigma)['delta'][0] -
                 get_all_greeks('c',i,k1,t2,0,sigma)['delta'][0] - get_all_greeks('p',i,k2 ,t2,0,sigma)['delta'][0])

    gamma.append(get_all_greeks('c',i,k1,t1,0,sigma)['gamma'][0] + get_all_greeks('p',i,k2 ,t1,0,sigma)['gamma'][0] -
                 get_all_greeks('c',i,k1,t2,0,sigma)['gamma'][0] - get_all_greeks('p',i,k2 ,t2,0,sigma)['gamma'][0])

    theta.append(get_all_greeks('c',i,k1,t1,0,sigma)['theta'][0] + get_all_greeks('p',i,k2 ,t1,0,sigma)['theta'][0] -
                 get_all_greeks('c',i,k1,t2,0,sigma)['theta'][0] - get_all_greeks('p',i,k2 ,t2,0,sigma)['theta'][0])

    vega.append(get_all_greeks('c',i,k1,t1,0,sigma)['vega'][0] + get_all_greeks('p',i,k2 ,t1, 0, sigma)['vega'][0] -
                get_all_greeks('c',i,k1,t2,0,sigma)['vega'][0] - get_all_greeks('p',i,k2 ,t2, 0, sigma)['vega'][0])

plot()

In [8]:
# 蝶式多头
# long ITM call + short OTM call X2 + long OTM call

s = 1000
k1,k2,k3 = 800,1000,1200
t = 0.5
sigma = 0.3

# pnl
pnl = []
for i in range(500,2000,100):
    pnl1 = (i - k1) if i > k1 else 0
    pnl2 = 0 if i < k2 else (k2-i) * 2
    pnl3 = (i - k3) if i > k3 else 0
    pl = pnl1 + pnl2 + pnl3
    pnl.append(pl)

# delta
delta=[]
# Gamma, Theta, Vega
gamma,theta,vega = [],[],[]
for i in range(500,2000,100):
    delta.append(get_all_greeks('c',i,k1,t,0,sigma)['delta'][0] - get_all_greeks('c',i,k2 ,t,0,sigma)['delta'][0] * 2 + get_all_greeks('c',i,k3,t,0,sigma)['delta'][0])
    gamma.append(get_all_greeks('c',i,k1,t,0,sigma)['gamma'][0] - get_all_greeks('c',i,k2 ,t,0,sigma)['gamma'][0] * 2 + get_all_greeks('c',i,k3,t,0,sigma)['gamma'][0])
    theta.append(get_all_greeks('c',i,k1,t,0,sigma)['theta'][0] - get_all_greeks('c',i,k2 ,t,0,sigma)['theta'][0] * 2 + get_all_greeks('c',i,k3,t,0,sigma)['theta'][0])
    vega.append(get_all_greeks('c',i,k1,t,0,sigma)['vega'][0] - get_all_greeks('c',i,k2 ,t,0,sigma)['vega'][0] * 2 + get_all_greeks('c',i,k3,t,0,sigma)['vega'][0])

plot()

In [9]:
# 蝶式空头
# short ITM call + long ATM call X2 + short OTM call

s = 1000
k1,k2,k3 = 800,1000,1200
t = 0.5
sigma = 0.3

# pnl
pnl = []
for i in range(500,2000,100):
    pnl1 = (k1 - i) if i > k1 else 0
    pnl2 = (i - k2) * 2 if i > k2 else 0
    pnl3 = (k3 - i) if i > k3 else 0
    pl = pnl1 + pnl2 + pnl3
    pnl.append(pl)

# delta
delta=[]
# Gamma, Theta, Vega
gamma,theta,vega = [],[],[]
for i in range(500,2000,100):
    delta.append(-get_all_greeks('c',i,k1,t,0,sigma)['delta'][0] + get_all_greeks('c',i,k2 ,t,0,sigma)['delta'][0] * 2 - get_all_greeks('c',i,k3,t,0,sigma)['delta'][0])
    gamma.append(-get_all_greeks('c',i,k1,t,0,sigma)['gamma'][0] + get_all_greeks('c',i,k2 ,t,0,sigma)['gamma'][0] * 2 - get_all_greeks('c',i,k3,t,0,sigma)['gamma'][0])
    theta.append(-get_all_greeks('c',i,k1,t,0,sigma)['theta'][0] + get_all_greeks('c',i,k2 ,t,0,sigma)['theta'][0] * 2 - get_all_greeks('c',i,k3,t,0,sigma)['theta'][0])
    vega.append(-get_all_greeks('c',i,k1,t,0,sigma)['vega'][0] + get_all_greeks('c',i,k2 ,t,0,sigma)['vega'][0] * 2 - get_all_greeks('c',i,k3,t,0,sigma)['vega'][0])

plot()

In [10]:
# condor
# long k1 put + short k2 put + short k3 call + long k4 call

s = 1000
k1,k2,k3,k4 = 800,1000,1200,1400
t = 0.5
sigma = 0.3

# pnl
pnl = []
for i in range(500,2000,100):
    pnl1 = (k1 - i) if i < k1 else 0
    pnl2 = (i - k2) if i < k2 else 0
    pnl3 = (k3 - i) if i > k3 else 0
    pnl4 = (i - k4) if i > k4 else 0
    pl = pnl1 + pnl2 + pnl3 + pnl4
    pnl.append(pl)

delta, gamma, theta, vega = [],[],[],[]
for i in range(500,2000,100):
    delta.append(get_all_greeks('p',i,k1,t,0,sigma)['delta'][0] - get_all_greeks('p',i,k2 ,t,0,sigma)['delta'][0] - get_all_greeks('c',i,k3 ,t,0,sigma)['delta'][0] + get_all_greeks('c',i,k4 ,t,0,sigma)['delta'][0])

    gamma.append(get_all_greeks('p',i,k1,t,0,sigma)['gamma'][0] - get_all_greeks('p',i,k2 ,t,0,sigma)['gamma'][0] - get_all_greeks('c',i,k3,t,0,sigma)['gamma'][0] + get_all_greeks('c',i,k4,t,0,sigma)['gamma'][0])

    theta.append(get_all_greeks('p',i,k1,t,0,sigma)['theta'][0] - get_all_greeks('p',i,k2 ,t,0,sigma)['theta'][0] - get_all_greeks('c',i,k3 ,t,0,sigma)['theta'][0] + get_all_greeks('c',i,k4 ,t,0,sigma)['theta'][0])

    vega.append(get_all_greeks('p',i,k1,t,0,sigma)['vega'][0] - get_all_greeks('p',i,k2 ,t,0,sigma)['vega'][0] - get_all_greeks('c',i,k3 ,t,0,sigma)['vega'][0] + get_all_greeks('c',i,k4 ,t,0,sigma)['vega'][0])

plot()

In [11]:
# condor空头
s = 1000
k1,k2,k3,k4 = 800,1000,1200,1400
t = 0.5
sigma = 0.3

# pnl
pnl = []
for i in range(500,2000,100):
    pnl1 = -(k1 - i) if i < k1 else 0
    pnl2 = -(i - k2) if i < k2 else 0
    pnl3 = -(k3 - i) if i > k3 else 0
    pnl4 = -(i - k4) if i > k4 else 0
    pl = pnl1 + pnl2 + pnl3 + pnl4
    pnl.append(pl)

delta, gamma, theta, vega = [],[],[],[]
for i in range(500,2000,100):
    delta.append(-get_all_greeks('p',i,k1,t,0,sigma)['delta'][0] + get_all_greeks('p',i,k2 ,t,0,sigma)['delta'][0] + get_all_greeks('c',i,k3 ,t,0,sigma)['delta'][0] - get_all_greeks('c',i,k4 ,t,0,sigma)['delta'][0])

    gamma.append(-get_all_greeks('p',i,k1,t,0,sigma)['gamma'][0] + get_all_greeks('p',i,k2 ,t,0,sigma)['gamma'][0] + get_all_greeks('c',i,k3,t,0,sigma)['gamma'][0] - get_all_greeks('c',i,k4,t,0,sigma)['gamma'][0])

    theta.append(-get_all_greeks('p',i,k1,t,0,sigma)['theta'][0] + get_all_greeks('p',i,k2 ,t,0,sigma)['theta'][0] + get_all_greeks('c',i,k3 ,t,0,sigma)['theta'][0] - get_all_greeks('c',i,k4 ,t,0,sigma)['theta'][0])

    vega.append(-get_all_greeks('p',i,k1,t,0,sigma)['vega'][0] + get_all_greeks('p',i,k2 ,t,0,sigma)['vega'][0] + get_all_greeks('c',i,k3 ,t,0,sigma)['vega'][0] - get_all_greeks('c',i,k4 ,t,0,sigma)['vega'][0])

plot()

In [12]:
# time butterfly

# 时间蝶式空头
# short ITM call + long ATM call X2 + short OTM call

s = 1000
k1,k2,k3 = 800,1000,1200
t1,t2,t3 = 0.5,0.6,0.7
sigma = 0.3

# pnl
pnl = []
for i in range(500,2000,100):
    pnl1 = (k1 - i) if i > k1 else 0
    pnl2 = (i - k2) * 2 if i > k2 else 0
    pnl3 = (k3 - i) if i > k3 else 0
    pl = pnl1 + pnl2 + pnl3
    pnl.append(pl)

# delta
delta=[]
# Gamma, Theta, Vega
gamma,theta,vega = [],[],[]
for i in range(500,2000,100):
    delta.append(-get_all_greeks('c',i,k1,t1,0,sigma)['delta'][0] + get_all_greeks('c',i,k2 ,t2,0,sigma)['delta'][0] * 2 - get_all_greeks('c',i,k3,t3,0,sigma)['delta'][0])
    gamma.append(-get_all_greeks('c',i,k1,t1,0,sigma)['gamma'][0] + get_all_greeks('c',i,k2 ,t2,0,sigma)['gamma'][0] * 2 - get_all_greeks('c',i,k3,t3,0,sigma)['gamma'][0])
    theta.append(-get_all_greeks('c',i,k1,t1,0,sigma)['theta'][0] + get_all_greeks('c',i,k2 ,t2,0,sigma)['theta'][0] * 2 - get_all_greeks('c',i,k3,t3,0,sigma)['theta'][0])
    vega.append(-get_all_greeks('c',i,k1,t1,0,sigma)['vega'][0] + get_all_greeks('c',i,k2 ,t2,0,sigma)['vega'][0] * 2 - get_all_greeks('c',i,k3,t3,0,sigma)['vega'][0])

plot()

In [ ]:
# 跨式多头 + 日历价差多头
# long t1 put + short t2 put + long t1 call + short t2 call

s = 1000
k1,k2 = 800,1200
t1,t2 = 0.5,0.8
sigma = 0.3


# Gamma, Theta, Vega
delta, gamma, theta, vega = [],[],[],[]
for i in range(500,2000,100):
    delta.append(get_all_greeks('c',i,k1,t1,0,sigma)['delta'][0] + get_all_greeks('p',i,k2 ,t1,0,sigma)['delta'][0] -
                 get_all_greeks('c',i,k1,t2,0,sigma)['delta'][0] - get_all_greeks('p',i,k2 ,t2,0,sigma)['delta'][0])

    gamma.append(get_all_greeks('c',i,k1,t1,0,sigma)['gamma'][0] + get_all_greeks('p',i,k2 ,t1,0,sigma)['gamma'][0] -
                 get_all_greeks('c',i,k1,t2,0,sigma)['gamma'][0] - get_all_greeks('p',i,k2 ,t2,0,sigma)['gamma'][0])

    theta.append(get_all_greeks('c',i,k1,t1,0,sigma)['theta'][0] + get_all_greeks('p',i,k2 ,t1,0,sigma)['theta'][0] -
                 get_all_greeks('c',i,k1,t2,0,sigma)['theta'][0] - get_all_greeks('p',i,k2 ,t2,0,sigma)['theta'][0])

    vega.append(get_all_greeks('c',i,k1,t1,0,sigma)['vega'][0] + get_all_greeks('p',i,k2 ,t1, 0, sigma)['vega'][0] -
                get_all_greeks('c',i,k1,t2,0,sigma)['vega'][0] - get_all_greeks('p',i,k2 ,t2, 0, sigma)['vega'][0])

plot()

In [ ]:
# 蝶式多头
# long ITM call + short OTM call X2 + long OTM call

s = 1000
k1,k2,k3 = 800,1000,1200
t = 0.5
sigma = 0.3

# pnl
pnl = []
for i in range(500,2000,100):
    pnl1 = (i - k1) if i > k1 else 0
    pnl2 = 0 if i < k2 else (k2-i) * 2
    pnl3 = (i - k3) if i > k3 else 0
    pl = pnl1 + pnl2 + pnl3
    pnl.append(pl)

# delta
delta=[]
# Gamma, Theta, Vega
gamma,theta,vega = [],[],[]
for i in range(500,2000,100):
    delta.append(get_all_greeks('c',i,k1,t,0,sigma)['delta'][0] - get_all_greeks('c',i,k2 ,t,0,sigma)['delta'][0] * 2 + get_all_greeks('c',i,k3,t,0,sigma)['delta'][0])
    gamma.append(get_all_greeks('c',i,k1,t,0,sigma)['gamma'][0] - get_all_greeks('c',i,k2 ,t,0,sigma)['gamma'][0] * 2 + get_all_greeks('c',i,k3,t,0,sigma)['gamma'][0])
    theta.append(get_all_greeks('c',i,k1,t,0,sigma)['theta'][0] - get_all_greeks('c',i,k2 ,t,0,sigma)['theta'][0] * 2 + get_all_greeks('c',i,k3,t,0,sigma)['theta'][0])
    vega.append(get_all_greeks('c',i,k1,t,0,sigma)['vega'][0] - get_all_greeks('c',i,k2 ,t,0,sigma)['vega'][0] * 2 + get_all_greeks('c',i,k3,t,0,sigma)['vega'][0])

plot()

In [ ]:
# 蝶式空头
# short ITM call + long ATM call X2 + short OTM call

s = 1000
k1,k2,k3 = 800,1000,1200
t = 0.5
sigma = 0.3

# pnl
pnl = []
for i in range(500,2000,100):
    pnl1 = (k1 - i) if i > k1 else 0
    pnl2 = (i - k2) * 2 if i > k2 else 0
    pnl3 = (k3 - i) if i > k3 else 0
    pl = pnl1 + pnl2 + pnl3
    pnl.append(pl)

# delta
delta=[]
# Gamma, Theta, Vega
gamma,theta,vega = [],[],[]
for i in range(500,2000,100):
    delta.append(-get_all_greeks('c',i,k1,t,0,sigma)['delta'][0] + get_all_greeks('c',i,k2 ,t,0,sigma)['delta'][0] * 2 - get_all_greeks('c',i,k3,t,0,sigma)['delta'][0])
    gamma.append(-get_all_greeks('c',i,k1,t,0,sigma)['gamma'][0] + get_all_greeks('c',i,k2 ,t,0,sigma)['gamma'][0] * 2 - get_all_greeks('c',i,k3,t,0,sigma)['gamma'][0])
    theta.append(-get_all_greeks('c',i,k1,t,0,sigma)['theta'][0] + get_all_greeks('c',i,k2 ,t,0,sigma)['theta'][0] * 2 - get_all_greeks('c',i,k3,t,0,sigma)['theta'][0])
    vega.append(-get_all_greeks('c',i,k1,t,0,sigma)['vega'][0] + get_all_greeks('c',i,k2 ,t,0,sigma)['vega'][0] * 2 - get_all_greeks('c',i,k3,t,0,sigma)['vega'][0])

plot()

In [ ]:
# condor
# long k1 put + short k2 put + short k3 call + long k4 call

s = 1000
k1,k2,k3,k4 = 800,1000,1200,1400
t = 0.5
sigma = 0.3

# pnl
pnl = []
for i in range(500,2000,100):
    pnl1 = (k1 - i) if i < k1 else 0
    pnl2 = (i - k2) if i < k2 else 0
    pnl3 = (k3 - i) if i > k3 else 0
    pnl4 = (i - k4) if i > k4 else 0
    pl = pnl1 + pnl2 + pnl3 + pnl4
    pnl.append(pl)

delta, gamma, theta, vega = [],[],[],[]
for i in range(500,2000,100):
    delta.append(get_all_greeks('p',i,k1,t,0,sigma)['delta'][0] - get_all_greeks('p',i,k2 ,t,0,sigma)['delta'][0] - get_all_greeks('c',i,k3 ,t,0,sigma)['delta'][0] + get_all_greeks('c',i,k4 ,t,0,sigma)['delta'][0])

    gamma.append(get_all_greeks('p',i,k1,t,0,sigma)['gamma'][0] - get_all_greeks('p',i,k2 ,t,0,sigma)['gamma'][0] - get_all_greeks('c',i,k3,t,0,sigma)['gamma'][0] + get_all_greeks('c',i,k4,t,0,sigma)['gamma'][0])

    theta.append(get_all_greeks('p',i,k1,t,0,sigma)['theta'][0] - get_all_greeks('p',i,k2 ,t,0,sigma)['theta'][0] - get_all_greeks('c',i,k3 ,t,0,sigma)['theta'][0] + get_all_greeks('c',i,k4 ,t,0,sigma)['theta'][0])

    vega.append(get_all_greeks('p',i,k1,t,0,sigma)['vega'][0] - get_all_greeks('p',i,k2 ,t,0,sigma)['vega'][0] - get_all_greeks('c',i,k3 ,t,0,sigma)['vega'][0] + get_all_greeks('c',i,k4 ,t,0,sigma)['vega'][0])

plot()

In [ ]:
# condor空头
s = 1000
k1,k2,k3,k4 = 800,1000,1200,1400
t = 0.5
sigma = 0.3

# pnl
pnl = []
for i in range(500,2000,100):
    pnl1 = -(k1 - i) if i < k1 else 0
    pnl2 = -(i - k2) if i < k2 else 0
    pnl3 = -(k3 - i) if i > k3 else 0
    pnl4 = -(i - k4) if i > k4 else 0
    pl = pnl1 + pnl2 + pnl3 + pnl4
    pnl.append(pl)

delta, gamma, theta, vega = [],[],[],[]
for i in range(500,2000,100):
    delta.append(-get_all_greeks('p',i,k1,t,0,sigma)['delta'][0] + get_all_greeks('p',i,k2 ,t,0,sigma)['delta'][0] + get_all_greeks('c',i,k3 ,t,0,sigma)['delta'][0] - get_all_greeks('c',i,k4 ,t,0,sigma)['delta'][0])

    gamma.append(-get_all_greeks('p',i,k1,t,0,sigma)['gamma'][0] + get_all_greeks('p',i,k2 ,t,0,sigma)['gamma'][0] + get_all_greeks('c',i,k3,t,0,sigma)['gamma'][0] - get_all_greeks('c',i,k4,t,0,sigma)['gamma'][0])

    theta.append(-get_all_greeks('p',i,k1,t,0,sigma)['theta'][0] + get_all_greeks('p',i,k2 ,t,0,sigma)['theta'][0] + get_all_greeks('c',i,k3 ,t,0,sigma)['theta'][0] - get_all_greeks('c',i,k4 ,t,0,sigma)['theta'][0])

    vega.append(-get_all_greeks('p',i,k1,t,0,sigma)['vega'][0] + get_all_greeks('p',i,k2 ,t,0,sigma)['vega'][0] + get_all_greeks('c',i,k3 ,t,0,sigma)['vega'][0] - get_all_greeks('c',i,k4 ,t,0,sigma)['vega'][0])

plot()

In [ ]:
# time butterfly

# 时间蝶式空头
# short ITM call + long ATM call X2 + short OTM call

s = 1000
k1,k2,k3 = 800,1000,1200
t1,t2,t3 = 0.5,0.6,0.7
sigma = 0.3

# pnl
pnl = []
for i in range(500,2000,100):
    pnl1 = (k1 - i) if i > k1 else 0
    pnl2 = (i - k2) * 2 if i > k2 else 0
    pnl3 = (k3 - i) if i > k3 else 0
    pl = pnl1 + pnl2 + pnl3
    pnl.append(pl)

# delta
delta=[]
# Gamma, Theta, Vega
gamma,theta,vega = [],[],[]
for i in range(500,2000,100):
    delta.append(-get_all_greeks('c',i,k1,t1,0,sigma)['delta'][0] + get_all_greeks('c',i,k2 ,t2,0,sigma)['delta'][0] * 2 - get_all_greeks('c',i,k3,t3,0,sigma)['delta'][0])
    gamma.append(-get_all_greeks('c',i,k1,t1,0,sigma)['gamma'][0] + get_all_greeks('c',i,k2 ,t2,0,sigma)['gamma'][0] * 2 - get_all_greeks('c',i,k3,t3,0,sigma)['gamma'][0])
    theta.append(-get_all_greeks('c',i,k1,t1,0,sigma)['theta'][0] + get_all_greeks('c',i,k2 ,t2,0,sigma)['theta'][0] * 2 - get_all_greeks('c',i,k3,t3,0,sigma)['theta'][0])
    vega.append(-get_all_greeks('c',i,k1,t1,0,sigma)['vega'][0] + get_all_greeks('c',i,k2 ,t2,0,sigma)['vega'][0] * 2 - get_all_greeks('c',i,k3,t3,0,sigma)['vega'][0])

plot()

In [14]:
# 蝶式多头
# long ITM call + short OTM call X2 + long OTM call

s = 1000
k1,k2,k3 = 800,1000,1200
t = 0.5
sigma = 0.3

# pnl
pnl = []
for i in range(500,2000,100):
    pnl1 = (i - k1) if i > k1 else 0
    pnl2 = 0 if i < k2 else (k2-i) * 2
    pnl3 = (i - k3) if i > k3 else 0
    pl = pnl1 + pnl2 + pnl3
    pnl.append(pl)

# delta
delta=[]
# Gamma, Theta, Vega
gamma,theta,vega = [],[],[]
for i in range(500,2000,100):
    delta.append(get_all_greeks('c',i,k1,t,0,sigma)['delta'][0] - get_all_greeks('c',i,k2 ,t,0,sigma)['delta'][0] * 2 + get_all_greeks('c',i,k3,t,0,sigma)['delta'][0])
    gamma.append(get_all_greeks('c',i,k1,t,0,sigma)['gamma'][0] - get_all_greeks('c',i,k2 ,t,0,sigma)['gamma'][0] * 2 + get_all_greeks('c',i,k3,t,0,sigma)['gamma'][0])
    theta.append(get_all_greeks('c',i,k1,t,0,sigma)['theta'][0] - get_all_greeks('c',i,k2 ,t,0,sigma)['theta'][0] * 2 + get_all_greeks('c',i,k3,t,0,sigma)['theta'][0])
    vega.append(get_all_greeks('c',i,k1,t,0,sigma)['vega'][0] - get_all_greeks('c',i,k2 ,t,0,sigma)['vega'][0] * 2 + get_all_greeks('c',i,k3,t,0,sigma)['vega'][0])

plot()

In [15]:
# 蝶式空头
# short ITM call + long ATM call X2 + short OTM call

s = 1000
k1,k2,k3 = 800,1000,1200
t = 0.5
sigma = 0.3

# pnl
pnl = []
for i in range(500,2000,100):
    pnl1 = (k1 - i) if i > k1 else 0
    pnl2 = (i - k2) * 2 if i > k2 else 0
    pnl3 = (k3 - i) if i > k3 else 0
    pl = pnl1 + pnl2 + pnl3
    pnl.append(pl)

# delta
delta=[]
# Gamma, Theta, Vega
gamma,theta,vega = [],[],[]
for i in range(500,2000,100):
    delta.append(-get_all_greeks('c',i,k1,t,0,sigma)['delta'][0] + get_all_greeks('c',i,k2 ,t,0,sigma)['delta'][0] * 2 - get_all_greeks('c',i,k3,t,0,sigma)['delta'][0])
    gamma.append(-get_all_greeks('c',i,k1,t,0,sigma)['gamma'][0] + get_all_greeks('c',i,k2 ,t,0,sigma)['gamma'][0] * 2 - get_all_greeks('c',i,k3,t,0,sigma)['gamma'][0])
    theta.append(-get_all_greeks('c',i,k1,t,0,sigma)['theta'][0] + get_all_greeks('c',i,k2 ,t,0,sigma)['theta'][0] * 2 - get_all_greeks('c',i,k3,t,0,sigma)['theta'][0])
    vega.append(-get_all_greeks('c',i,k1,t,0,sigma)['vega'][0] + get_all_greeks('c',i,k2 ,t,0,sigma)['vega'][0] * 2 - get_all_greeks('c',i,k3,t,0,sigma)['vega'][0])

plot()

In [16]:
# condor
# long k1 put + short k2 put + short k3 call + long k4 call

s = 1000
k1,k2,k3,k4 = 800,1000,1200,1400
t = 0.5
sigma = 0.3

# pnl
pnl = []
for i in range(500,2000,100):
    pnl1 = (k1 - i) if i < k1 else 0
    pnl2 = (i - k2) if i < k2 else 0
    pnl3 = (k3 - i) if i > k3 else 0
    pnl4 = (i - k4) if i > k4 else 0
    pl = pnl1 + pnl2 + pnl3 + pnl4
    pnl.append(pl)

delta, gamma, theta, vega = [],[],[],[]
for i in range(500,2000,100):
    delta.append(get_all_greeks('p',i,k1,t,0,sigma)['delta'][0] - get_all_greeks('p',i,k2 ,t,0,sigma)['delta'][0] - get_all_greeks('c',i,k3 ,t,0,sigma)['delta'][0] + get_all_greeks('c',i,k4 ,t,0,sigma)['delta'][0])

    gamma.append(get_all_greeks('p',i,k1,t,0,sigma)['gamma'][0] - get_all_greeks('p',i,k2 ,t,0,sigma)['gamma'][0] - get_all_greeks('c',i,k3,t,0,sigma)['gamma'][0] + get_all_greeks('c',i,k4,t,0,sigma)['gamma'][0])

    theta.append(get_all_greeks('p',i,k1,t,0,sigma)['theta'][0] - get_all_greeks('p',i,k2 ,t,0,sigma)['theta'][0] - get_all_greeks('c',i,k3 ,t,0,sigma)['theta'][0] + get_all_greeks('c',i,k4 ,t,0,sigma)['theta'][0])

    vega.append(get_all_greeks('p',i,k1,t,0,sigma)['vega'][0] - get_all_greeks('p',i,k2 ,t,0,sigma)['vega'][0] - get_all_greeks('c',i,k3 ,t,0,sigma)['vega'][0] + get_all_greeks('c',i,k4 ,t,0,sigma)['vega'][0])

plot()

In [17]:
# condor空头
s = 1000
k1,k2,k3,k4 = 800,1000,1200,1400
t = 0.5
sigma = 0.3

# pnl
pnl = []
for i in range(500,2000,100):
    pnl1 = -(k1 - i) if i < k1 else 0
    pnl2 = -(i - k2) if i < k2 else 0
    pnl3 = -(k3 - i) if i > k3 else 0
    pnl4 = -(i - k4) if i > k4 else 0
    pl = pnl1 + pnl2 + pnl3 + pnl4
    pnl.append(pl)

delta, gamma, theta, vega = [],[],[],[]
for i in range(500,2000,100):
    delta.append(-get_all_greeks('p',i,k1,t,0,sigma)['delta'][0] + get_all_greeks('p',i,k2 ,t,0,sigma)['delta'][0] + get_all_greeks('c',i,k3 ,t,0,sigma)['delta'][0] - get_all_greeks('c',i,k4 ,t,0,sigma)['delta'][0])

    gamma.append(-get_all_greeks('p',i,k1,t,0,sigma)['gamma'][0] + get_all_greeks('p',i,k2 ,t,0,sigma)['gamma'][0] + get_all_greeks('c',i,k3,t,0,sigma)['gamma'][0] - get_all_greeks('c',i,k4,t,0,sigma)['gamma'][0])

    theta.append(-get_all_greeks('p',i,k1,t,0,sigma)['theta'][0] + get_all_greeks('p',i,k2 ,t,0,sigma)['theta'][0] + get_all_greeks('c',i,k3 ,t,0,sigma)['theta'][0] - get_all_greeks('c',i,k4 ,t,0,sigma)['theta'][0])

    vega.append(-get_all_greeks('p',i,k1,t,0,sigma)['vega'][0] + get_all_greeks('p',i,k2 ,t,0,sigma)['vega'][0] + get_all_greeks('c',i,k3 ,t,0,sigma)['vega'][0] - get_all_greeks('c',i,k4 ,t,0,sigma)['vega'][0])

plot()

In [18]:
# time butterfly

# 时间蝶式空头
# short ITM call + long ATM call X2 + short OTM call

s = 1000
k1,k2,k3 = 800,1000,1200
t1,t2,t3 = 0.5,0.6,0.7
sigma = 0.3

# pnl
pnl = []
for i in range(500,2000,100):
    pnl1 = (k1 - i) if i > k1 else 0
    pnl2 = (i - k2) * 2 if i > k2 else 0
    pnl3 = (k3 - i) if i > k3 else 0
    pl = pnl1 + pnl2 + pnl3
    pnl.append(pl)

# delta
delta=[]
# Gamma, Theta, Vega
gamma,theta,vega = [],[],[]
for i in range(500,2000,100):
    delta.append(-get_all_greeks('c',i,k1,t1,0,sigma)['delta'][0] + get_all_greeks('c',i,k2 ,t2,0,sigma)['delta'][0] * 2 - get_all_greeks('c',i,k3,t3,0,sigma)['delta'][0])
    gamma.append(-get_all_greeks('c',i,k1,t1,0,sigma)['gamma'][0] + get_all_greeks('c',i,k2 ,t2,0,sigma)['gamma'][0] * 2 - get_all_greeks('c',i,k3,t3,0,sigma)['gamma'][0])
    theta.append(-get_all_greeks('c',i,k1,t1,0,sigma)['theta'][0] + get_all_greeks('c',i,k2 ,t2,0,sigma)['theta'][0] * 2 - get_all_greeks('c',i,k3,t3,0,sigma)['theta'][0])
    vega.append(-get_all_greeks('c',i,k1,t1,0,sigma)['vega'][0] + get_all_greeks('c',i,k2 ,t2,0,sigma)['vega'][0] * 2 - get_all_greeks('c',i,k3,t3,0,sigma)['vega'][0])

plot()